Before you turn this problem in, make sure everything runs as expected. First, **restart the kernel** (in the menubar, select Kernel$\rightarrow$Restart) and then **run all cells** (in the menubar, select Cell$\rightarrow$Run All).

Make sure you fill in any place that says `YOUR CODE HERE`, as well as your name below:

In [ ]:
NAME = ""

Note: some of the cells below are locked, so you cannot change them, but you can evaluate them!

---

# Computational Methods in Economics (WiSe 2019/20)

## Exam Question

#### DEADLINE: Tuesday, February 11, 6 pm (18:00)

Please submit your answer in the same way as the problem sets: save this notebook in a folder of the same name (`CME_Final`) and push it to your LRZ-Gitlab repository. Do not change the name of the notebook!

In [ ]:
import numpy as np
import pandas as pd
import scipy.optimize

import matplotlib.pyplot as plt
%matplotlib inline

import funapprox_cme as fa 

Consider an endowment economy with $m$ agents and $n$ goods. Throughout this question, *superscripts* will indicate agents, while *subscripts* will indicate goods.

Agent $i$'s utility function over the $n$ goods is given by:

\begin{equation}
    u^{i}(\mathbf{x}^i) = \sum^{n}_{j = 1} a^i_j (x^i_j)^{v^i_j + 1} (1 + v^i_j)^{-1} 
\end{equation}

Suppose that agent $i$'s endowment of good $j$ is $e^i_j$. Assume that $a^i_j, e^i_j > 0$ and $v^i_j < 0$ (for $v^i_j =-1$, we replace $(x^i_j)^{v^i_j + 1} (1 + v^i_j)^{-1}$ with $\ln x^i_j$). 

The parameters $v^i_j$, $a^i_j$ and $e^i_j$ are given as Numpy arrays. For example, for a setting with $m = 3$ (three agents) and $n = 2$ (two goods):

In [ ]:
## read parameters
A = np.array([ [2.0, 1.5],
               [1.5, 2.0],
               [1.5, 2.0]  ])

V = np.array([ [-2.0, -0.5],
               [-1.5, -0.5],
               [-0.5, -1.5] ])

E = np.array([ [2.0, 3.0],
               [1.0, 2.0],
               [4.0, 0.0]  ])


The way to read these matrices is that an agent corresponds to a row and a good to a column. For example, agent 1's endowment of good 2, $e^1_2$, would be the element in the first row and second column of matrix **E**, and hence $e^1_2 = 3.0$.

-------------------------------------------------------------------------------------------------------------------------------

(a) In this question, we use *numerical optimization* to solve for the outcome of the planning problem, in which the planner maximizes total welfare:

$$
    \max_{\left\{\mathbf{x}^i\right\}_{i = 1}^m} \sum_{i = 1}^m \lambda^i u^{i}(\mathbf{x}^i) \tag{1}
$$

subject to the resource constraint, i.e. aggregate consumption of good $j$ must equal aggregate endowments:

<a id = 'rc'></a>
\begin{equation}
    \sum^{m}_{i = 1} x_j^i = \sum^{m}_{i = 1} e_j^i, \quad \forall j \tag{RC}
\end{equation}

where $\lambda^i$ is the social weight associated with agent $i$.



Write a function **solve_planner** that takes as inputs an array **x0**, the arrays **V**, **A** and **E** (each of them with dimension $m$-by-$n$), and a flat array **lam** with length $m$ containing the social weights $\lambda^1,...,\lambda^m$ (compare equation (1)), and returns the solution to the social planner's problem. That is, it should return an $m$-by-$n$ Numpy array **X**, where element $x^i_{j}$ is the planner's allocation of good $j$ to agent $i$. **x0** contains the initial guess, and can be either flat (with length $m \cdot n$) or of dimension $m$-by-$n$.

You can use *either constrained or unconstrained numerical optimization*, and make use of any built-in Python function. Note that your code should be written (and will be tested!) in a way that it works for any numbers $m$ and $n$.


**Hint**: A slightly tricky issue when answering this question using *unconstrained* numerical optimization methods is how to deal with the constraint 

\begin{equation}
    \sum^{n}_{i = 1} x_j^i = \sum^{n}_{i = 1} e_j^i
\end{equation}

One way to address this is to have the algorithm solve for the optimal consumption for $m - 1$ agents and evaluate the consumption and hence the utility of the last agent *as the residual*. Formally, for good $j$,

\begin{equation}
    x_j^m = \sum^{m}_{i = 1} e_j^i - \sum^{m-1}_{i = 1} x_j^i
\end{equation}




In [ ]:
# YOUR CODE HERE

In [ ]:
## test case from PS4

A = np.array([ [2.0, 1.5, 1.5],
               [1.5, 2.0, 1.5]])

V = np.array([ [-2.0, -0.5, -0.5],
               [-1.5, -0.5, -1.5]])

E = np.array([ [2.0, 3.0, 0.0],
               [1.0, 2.0, 4.0]])

x0 = np.array([2., 2., 2., 1., 3., 2.])
lam1 = 0.5
lam = np.array([lam1, 1-lam1])
assert np.allclose( solve_planner(x0, V, A, E, lam), np.array([[1.53677822, 1.79949481, 2.62096144], 
                                                               [1.46322178, 3.20050519, 1.37903856]]), atol = 1e-2 )

In [ ]:
## THIS IS A TEST CELL!

-------------------------------------------------------------------------------------------------------------------------------

(b) Note that the planning solution does not depend on how initial endowments are distributed between agents; all that matters are the total endowments (across agents) of goods. To illustrate this, consider the case with $m = 3$ agents and $n = 2$ goods. Here, we are interested in how agent 3's consumption of good 1, $x^3_1$, changes compared to agent 1's consumption $x^1_1$, when agent 1's endowment of good 1, $e^1_1$, increases. In other words, we want to find (that is, approximate) the functions $x^1_1 = \pi(e_1^1)$ and $x^3_1 = \phi(e_1^1)$. 

Start with the following endowment matrix:

In [ ]:
E0 = np.array([ [0.0, 3.0],
               [1.0, 2.0],
               [4.0, 0.0]  ])

To find your data points, you should only vary the first element (**E[0,0]**). Keep all other values constant. Approximate the functions using 9 grid points, making use of your **solve_planner** function from question (a), for the inputs given below. Hence, you should return Numpy arrays **a_11** and **a_31** of length 9 that contain the corresponding approximation coefficients. Plot the approximated functions along a dense grid, and add labels and a legend to your plot. 

In [ ]:
A = np.array([ [2.0, 1.5],
               [1.5, 2.0],
               [1.5, 2.0]  ])

V = np.array([ [-2.0, -0.5],
               [-1.5, -0.5],
               [-0.5, -1.5] ])


X0 = np.array([[1., 2.], [2., 2.], [2., 1.]])  
lam1, lam2 = 0.5595, 0.324
lam = np.array([lam1, lam2, 1-lam1-lam2])   

In [ ]:
# YOUR CODE HERE

In [ ]:
## THIS IS A TEST CELL!

-------------------------------------------------------------------------------------------------------------------------------
(c) In this question, we use *root finding* to solve for a market equilibrium.  

The problem of agent $i$ is given by

\begin{equation}
    \max_{x^i}\ u^{i}(x^i) = \sum^{n}_{j = 1} a^i_j (x^i_j)^{v^i_j + 1} (1 + v^i_j)^{-1} 
\end{equation}

s.t. $\sum^{n}_{j = 1} p_j x^i_{j} = \sum^{n}_{j = 1} p_j e^i_{j} $, where $p_j$ denotes the market price of good $j$. 

Taking first-conditions gives 

\begin{equation}
    \frac{\partial u^i(x^i)}{\partial x^i_{j}} = u^i_{j}(x^i) = a^i_{j} (x^i_{j})^{v^i_{j}} = \mu_i p_j,\quad \forall i,j \tag{2}
\end{equation},

where $\mu_i$ is the Lagrange multiplier corresponding to agent $i$.

Consider $n = 2$, and normalize prices such that $p_1 + p_2 = 1$. Equation (2) then results in one first-order conditions for each agent $i$:

\begin{equation}
     \frac{ a^i_{1} (x^i_{1})^{v^i_{1}} }{p_1} - \frac{ a^i_{2} (x^i_{2})^{v^i_{2}} }{p_2} = 0
\end{equation}

Moreover, we have one budget constraint per agent $i$:

\begin{equation}
    p_1 x^i_{1} + p_2 x^i_{2} - p_1 e^i_{1} - p_2 e^i_{2} = 0
\end{equation}

Finally, the markets clear, implying two resource constraints:

\begin{equation}
    \sum_{i} x^i_{1} = \sum_{i}  e^i_{1},\quad \sum_{i} x^i_{2} = \sum_{i}  e^i_{2}
\end{equation}

In total, we have a system of $2m + 2$ equations - $m$ f.o.c.'s, $m$ budget constraints and two resource constraints - and $2m + 1$ unknown variables, $2m$ quantities $x^i_{j}$ and the price $p_1$ (recall that $p_2 = 1 - p_1$). 

Of course, Walras' law applies: in the context of this question, given that the agents' budget constraints all hold with equality, if we have an equilibrium on one market (i.e. no excess demand or supply), we must also have an equilibrium on the other market. Hence, we have to use only 7 of the 8 equations, without changing the problem. 


Write a function **S** that implements the system of non-linear equations outlined above. It takes a flat Numpy array **z** of length 7, as well as the arrays **V**, **A**, and **E** as inputs. Following the explanation above, omit market clearing for good 2 from your system. 

Note that since the algorithm should not consider negative values, you should use the "trick" with solving for the logarithms and using the exponential function. That is, element $z^i_j$ of your vector **z** is given by $z^i_j = \log(x^i_j)$.


Then, use write a function **solve_market** that takes the same arguments as **S** and uses Broyden's method to solve the system for the market equilibrium. It should return a *tuple*, where the first element is an array of dimension $m$-by-$2$ that contains the consumption quantities $x^i_j$, and the second element is the price $p_1$ (a scalar). 

**Hint**: Comparing the planning solution in question (a) and the market equilibrium (c), since there are no externalities or other frictions in this economy, the welfare theorems hold; hence, both setups will lead to the same allocations, as long as the social weights $\lambda_i$ and the initial endowments are chosen accordingly.

The following scenario should lead to very similar consumption arrays in both (a) and (c):

In [ ]:
lam1, lam2 = 0.5595, 0.324
lam = np.array([lam1, lam2, 1-lam1-lam2]) 


A = np.array([ [2.0, 1.5],
               [1.5, 2.0],
               [1.5, 2.0]  ])

V = np.array([ [-2.0, -0.5],
               [-1.5, -0.5],
               [-0.5, -1.5] ])

E = np.array([ [2.0, 3.0],
               [1.0, 2.0],
               [4.0, 0.0]  ])



In [ ]:
def S(z, V, A, E):
    """
    Implements the system of equations to solve for the market equilibrium for m agents and two goods
    
    ((2m + 1,) np.array, (m,2) np.array, (m,2) np.array, (m,2) np.array) -> (2m + 1,) np.array
    """
    # YOUR CODE HERE

def solve_market(z0, V, A, E):
    """
    Solves the system of equations to solve for the market equilibrium for m agents and two goods
    
    ((2m + 1,) np.array, (m,2) np.array, (m,2) np.array, (m,2) np.array) -> ( (m,2) np.array, float)
    """
    # YOUR CODE HERE

In [ ]:
x = np.ones(7)
assert np.allclose( S(x, V, A, E), np.array([ 0.62905421,  0.82910093,  0.59440835,  2.43656366,  3.43656366,
       -8.15484549,  1.15484549]))

In [ ]:
## THIS IS A TEST CELL!